# Two-sided Wilcoxon SIGNED test - variance

By: Sam<br>
Updated at: 27/04/23<br>
Compare performance of discretizers <br>

===

Matched pairs settings
- Sample: 540 ML models after discretization (as at 27/04/2023)
- Purpose: pair-wise comparison metrics of the models using different discretization method
    - Test 1.1: Compare variance between pairs of discretizers, regardless of algorithms
    - Test 1.2-1.6: Compare variance between pairs of discretizers, filter by models (CNB, ID3, Knn-Hamming, Knn-VDM, KNN)
    
===

Input data: instrinsic properties and model performance metrics <br>
ChiMerge manuallly and ChiMerge-SB are merged together. <br>
!!! **NB: Please update the data for metrics and export to csv before running this script!

In [1]:
# Import library
import pandas as pd
import numpy as np
from scipy import stats
import math
import random

In [2]:
# Import evaluation data (updated at 27/04/2023)
data = pd.read_csv("all_evaluation_270423.csv")

In [3]:
data.head()

,dataset,disc,param,inconsistency,models,con_features,size,time_disc,accuracy,time_train,bias,variance
0,iris,EWD,4,0.0667,ID3,4,150,0.0164,0.84,0.008698225,0.158,0.055
1,iris,EWD,7,0.02,ID3,4,150,0.0157,0.79,0.010634899,0.158,0.054
2,iris,EWD,10,0.0067,ID3,4,150,0.0164,0.95,0.010643005,0.053,0.014
3,iris,EFD,4,0.04,ID3,4,150,0.0167,0.84,0.009439945,0.158,0.049
4,iris,EFD,7,0.04,ID3,4,150,0.0239,0.95,0.010675907,0.053,0.07


In [4]:
data['disc'].unique() # get list of discretizers

array(['EWD', 'EFD', 'FFD', 'ChiMerge', 'DT', 'ChiMerge-SB'], dtype=object)

In [5]:
data.columns

Index(['dataset', 'disc', 'param', ' inconsistency ', 'models', 'con_features',
       'size', 'time_disc', 'accuracy', 'time_train', 'bias', 'variance'],
      dtype='object')

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 540 entries, 0 to 539
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   dataset          540 non-null    object 
 1   disc             540 non-null    object 
 2   param            540 non-null    int64  
 3    inconsistency   540 non-null    object 
 4   models           540 non-null    object 
 5   con_features     540 non-null    int64  
 6   size             540 non-null    object 
 7   time_disc        540 non-null    float64
 8   accuracy         540 non-null    object 
 9   time_train       540 non-null    object 
 10  bias             540 non-null    object 
 11  variance         540 non-null    object 
dtypes: float64(1), int64(2), object(9)
memory usage: 50.8+ KB


In [7]:
data['models'].unique()

array(['ID3', 'CNB', 'Knn-VDM', 'Knn-Hamming'], dtype=object)

# Wilcoxon_Complete pipeline

## 1.  Wilcoxon signed t test, variance
Ref: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.wilcoxon.html
Implement 4 replications: (DONE)
- Regardless algorithms
- Filter for each algorithm: CNB, ID3, Knn

## 1.1 variance, no filter in algorithm

In [8]:
# Preparation: Prepare list of metrics for each discretization (test_list)
# Step 1: Obtain variance for each discretization, convert into numeric, string values will be return as NaN
ewd_acc = pd.to_numeric(data[data['disc']=="EWD"]['variance'],errors='coerce').tolist()
efd_acc = pd.to_numeric(data[data['disc']=="EFD"]['variance'],errors='coerce').tolist()
ffd_acc = pd.to_numeric(data[data['disc']=="FFD"]['variance'],errors='coerce').tolist()
cm_acc = pd.to_numeric(data[(data['disc']=="ChiMerge") | (data['disc'] == "ChiMerge-SB")]['variance'],errors='coerce').tolist()
dt_acc = pd.to_numeric(data[data['disc']=="DT"]['variance'],errors='coerce').tolist()
# Check number of metrics available for each discretizer
print(len(ewd_acc))
print(len(efd_acc))
print(len(ffd_acc))
print(len(cm_acc))
print(len(dt_acc))

# Step 2: filter numeric values
raw_list = [ewd_acc, efd_acc, ffd_acc, cm_acc, dt_acc]
num_list = [] # filter numeric values only
for metric in raw_list:
    metric_new = [x for x in metric if (math.isnan(x) == False)] # using list comprehension
    num_list.append(metric_new)
    
# Step 3: random sample
# For discretization methods with different value, randomly select so that the final sample size are equal
# Reference: https://docs.python.org/3/library/random.html
k = min(len(metric) for metric in num_list)
test_list = []
for metric in num_list:
    random.seed(20)
    if len(metric) > k:
        metric = random.sample(metric, k=k)
    else: metric = metric
    #print(metric)
    test_list.append(metric)

90
90
120
120
120


In [9]:
# Initialiation
disc_key = ['ewd', 'efd', 'ffd', 'cm', 'dt']
disc_value = test_list # list of metrics for each discretization after preparation
test_stat = []
p_value = []
disc_compare = []
# Create dictionary store discretization and series of variance
disc = {}
for key in disc_key:
    for value in disc_value:
        disc[key] = value
        disc_value.remove(value)
        break  

# Create loop for Wilcoxon test (two sided) - handle diff = 0
for i in disc_key:
    for j in disc_key:
        if i != j:
            disc_compare.append(f'{i} vs {j}')
            # print(f'{i} vs {j}')
            # Compute difference list
            diff_list = []
            for m in range(0, len(disc[i])):
                diff = disc[i][m] - disc[j][m]
                diff_list.append(diff)
                                
            if all(item == 0 for item in diff_list) == False: # if the diff list does not contain all 0
                test_stat.append(stats.wilcoxon(diff_list).statistic)
                p_value.append(stats.wilcoxon(diff_list).pvalue)
            else: # if the diff list contain only0, cannot do Wilcoxon test
                error = 'N/A'
                test_stat.append(error)
                p_value.append(error)

In [10]:
# Convert 3 lists to dataframe
disc_compare = pd.DataFrame(disc_compare, columns=['disc_compare'])
test_stat = pd.DataFrame(test_stat, columns=['wtest_stat'])
p_value = pd.DataFrame(p_value, columns=['p_value'])

In [11]:
# Result table
wt_result = pd.concat([disc_compare, test_stat, p_value], axis = 1)
# Drop duplicate
wt_result.drop_duplicates(subset=['p_value'], inplace=True)

In [12]:
wt_result

,disc_compare,wtest_stat,p_value
0,ewd vs efd,1486.5,0.647710
1,ewd vs ffd,1153.0,0.036906
2,ewd vs cm,1540.0,0.998013
3,ewd vs dt,1533.5,0.972187
5,efd vs ffd,1120.0,0.024571
6,efd vs cm,1472.0,0.597628
7,efd vs dt,1518.5,0.763751
10,ffd vs cm,1153.0,0.036905
11,ffd vs dt,1099.5,0.028054
15,cm vs dt,1408.5,0.930567


In [13]:
# Exporting result

model = 'all'
metric = 'variance'
test = 'two_sided'

filename = f"wilcoxon_{metric}_{model}-{test}.csv"
wt_result['model'] = model
wt_result['metric'] = metric

wt_result.to_csv(filename, index = False)

## 1.2 variance, only CNB

In [14]:
# Preparation: Prepare list of metrics for each discretization (test_list)
# Step 1: Obtain variance for each discretization, convert into numeric, string values will be return as NaN
# Filter for CNB
# Obtain variance for each discretization, convert into numeric, string values will be return as NaN
ewd_acc_cnb = pd.to_numeric(data[(data['disc']=="EWD") & (data['models']=="CNB")]['variance'],errors='coerce').tolist()
efd_acc_cnb = pd.to_numeric(data[(data['disc']=="EFD") & (data['models']=="CNB")]['variance'],errors='coerce').tolist()
ffd_acc_cnb = pd.to_numeric(data[(data['disc']=="FFD") & (data['models']=="CNB")]['variance'],errors='coerce').tolist()
cm_acc_cnb = pd.to_numeric(data[((data['disc']=="ChiMerge") | (data['disc'] == "ChiMerge-SB")) & (data['models']=="CNB")]['variance'],errors='coerce').tolist()
dt_acc_cnb = pd.to_numeric(data[(data['disc']=="DT") & (data['models']=="CNB")]['variance'],errors='coerce').tolist()
# Check number of metrics available for each discretizer
print(len(ewd_acc_cnb))
print(len(efd_acc_cnb))
print(len(ffd_acc_cnb))
print(len(cm_acc_cnb))
print(len(dt_acc_cnb))

# Step 2: filter numeric values
raw_list = [ewd_acc_cnb,efd_acc_cnb, ffd_acc_cnb, cm_acc_cnb, dt_acc_cnb]
num_list = [] # filter numeric values only
for metric in raw_list:
    metric_new = [x for x in metric if (math.isnan(x) == False)] # using list comprehension
    num_list.append(metric_new)
    
# Step 3: random sample
# For discretization methods with different value, randomly select so that the final sample size are equal
# Reference: https://docs.python.org/3/library/random.html
k = min(len(metric) for metric in num_list)
test_list = []
for metric in num_list:
    random.seed(20)
    if len(metric) > k:
        metric = random.sample(metric, k=k)
    else: metric = metric
    #print(metric)
    test_list.append(metric)

30
30
40
40
40


In [15]:
# Initialiation
disc_key = ['ewd', 'efd', 'ffd', 'cm', 'dt']
disc_value = test_list # list of metrics for each discretization after preparation
test_stat = []
p_value = []
disc_compare = []
# Create dictionary store discretization and series of variance
disc = {}
for key in disc_key:
    for value in disc_value:
        disc[key] = value
        disc_value.remove(value)
        break  

# Create loop for Wilcoxon test (two sided) - handle diff = 0
for i in disc_key:
    for j in disc_key:
        if i != j:
            disc_compare.append(f'{i} vs {j}')
            # print(f'{i} vs {j}')
            # Compute difference list
            diff_list = []
            for m in range(0, len(disc[i])):
                diff = disc[i][m] - disc[j][m]
                diff_list.append(diff)
                
            if all(item == 0 for item in diff_list) == False: # if the diff list does not contain all 0
                test_stat.append(stats.wilcoxon(diff_list).statistic)
                p_value.append(stats.wilcoxon(diff_list).pvalue)
            else: # if the diff list contain only0, cannot do Wilcoxon test
                error = 'N/A'
                test_stat.append(error)
                p_value.append(error)

/Users/anhtth/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/morestats.py:3141: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


In [16]:
# Convert 3 lists to dataframe
disc_compare = pd.DataFrame(disc_compare, columns=['disc_compare'])
test_stat = pd.DataFrame(test_stat, columns=['wtest_stat'])
p_value = pd.DataFrame(p_value, columns=['p_value'])

In [17]:
# Result table
wt_result = pd.concat([disc_compare, test_stat, p_value], axis = 1)
# Drop duplicate
wt_result.drop_duplicates(subset=['p_value'], inplace=True)

In [18]:
wt_result

,disc_compare,wtest_stat,p_value
0,ewd vs efd,114.0,0.958420
1,ewd vs ffd,34.0,0.001673
2,ewd vs cm,95.5,0.486564
3,ewd vs dt,85.0,0.289095
5,efd vs ffd,52.0,0.014095
6,efd vs cm,104.0,0.482591
7,efd vs dt,87.5,0.209928
10,ffd vs cm,65.0,0.046160
11,ffd vs dt,79.0,0.128880
15,cm vs dt,64.0,0.553756


In [19]:
# Exporting result

model = 'CNB'
metric = 'variance'
test = 'two_sided'

filename = f"wilcoxon_{metric}_{model}-{test}.csv"
wt_result['model'] = model
wt_result['metric'] = metric

wt_result.to_csv(filename, index = False)

## 1.3 variance, only ID3

In [20]:
# Preparation: Prepare list of metrics for each discretization (test_list)
# Step 1: Obtain variance for each discretization, convert into numeric, string values will be return as NaN
# Filter for ID3
# Obtain variance for each discretization, convert into numeric, string values will be return as NaN
ewd_acc_ID3 = pd.to_numeric(data[(data['disc']=="EWD") & (data['models']=="ID3")]['variance'],errors='coerce').tolist()
efd_acc_ID3 = pd.to_numeric(data[(data['disc']=="EFD") & (data['models']=="ID3")]['variance'],errors='coerce').tolist()
ffd_acc_ID3 = pd.to_numeric(data[(data['disc']=="FFD") & (data['models']=="ID3")]['variance'],errors='coerce').tolist()
cm_acc_ID3 = pd.to_numeric(data[((data['disc']=="ChiMerge") | (data['disc'] == "ChiMerge-SB")) & (data['models']=="ID3")]['variance'],errors='coerce').tolist()
dt_acc_ID3 = pd.to_numeric(data[(data['disc']=="DT") & (data['models']=="ID3")]['variance'],errors='coerce').tolist()
# Check number of metrics available for each discretizer
print(len(ewd_acc_ID3))
print(len(efd_acc_ID3))
print(len(ffd_acc_ID3))
print(len(cm_acc_ID3))
print(len(dt_acc_ID3))

# Step 2: filter numeric values
raw_list = [ewd_acc_ID3,efd_acc_ID3, ffd_acc_ID3, cm_acc_ID3, dt_acc_ID3]
num_list = [] # filter numeric values only
for metric in raw_list:
    metric_new = [x for x in metric if (math.isnan(x) == False)] # using list comprehension
    num_list.append(metric_new)
    
# Step 3: random sample
# For discretization methods with different value, randomly select so that the final sample size are equal
# Reference: https://docs.python.org/3/library/random.html
k = min(len(metric) for metric in num_list)
test_list = []
for metric in num_list:
    random.seed(20)
    if len(metric) > k:
        metric = random.sample(metric, k=k)
    else: metric = metric
    #print(metric)
    test_list.append(metric)

30
30
40
40
40


In [21]:
# Initialiation
disc_key = ['ewd', 'efd', 'ffd', 'cm', 'dt']
disc_value = test_list # list of metrics for each discretization after preparation
test_stat = []
p_value = []
disc_compare = []
# Create dictionary store discretization and series of variance
disc = {}
for key in disc_key:
    for value in disc_value:
        disc[key] = value
        disc_value.remove(value)
        break  

# Create loop for Wilcoxon test (two sided) - handle diff = 0
for i in disc_key:
    for j in disc_key:
        if i != j:
            disc_compare.append(f'{i} vs {j}')
            # print(f'{i} vs {j}')
            # Compute difference list
            diff_list = []
            for m in range(0, len(disc[i])):
                diff = disc[i][m] - disc[j][m]
                diff_list.append(diff)
                              
            if all(item == 0 for item in diff_list) == False: # if the diff list does not contain all 0
                test_stat.append(stats.wilcoxon(diff_list).statistic)
                p_value.append(stats.wilcoxon(diff_list).pvalue)
            else: # if the diff list contain only0, cannot do Wilcoxon test
                error = 'N/A'
                test_stat.append(error)
                p_value.append(error)

In [22]:
# Convert 3 lists to dataframe
disc_compare = pd.DataFrame(disc_compare, columns=['disc_compare'])
test_stat = pd.DataFrame(test_stat, columns=['wtest_stat'])
p_value = pd.DataFrame(p_value, columns=['p_value'])

In [23]:
# Result table
wt_result = pd.concat([disc_compare, test_stat, p_value], axis = 1)
# Drop duplicate
wt_result.drop_duplicates(subset=['p_value'], inplace=True)

In [24]:
wt_result

,disc_compare,wtest_stat,p_value
0,ewd vs efd,67.5,0.002029
1,ewd vs ffd,177.5,0.257934
2,ewd vs cm,203.0,0.544006
3,ewd vs dt,183.0,0.308602
5,efd vs ffd,208.0,0.837239
6,efd vs cm,218.0,0.765519
7,efd vs dt,228.0,0.926247
10,ffd vs cm,100.5,0.056770
11,ffd vs dt,189.5,0.544857
15,cm vs dt,213.0,0.688344


In [25]:
# Exporting result

model = 'ID3'
metric = 'variance'
test = 'two_sided'

filename = f"wilcoxon_{metric}_{model}-{test}.csv"
wt_result['model'] = model
wt_result['metric'] = metric

wt_result.to_csv(filename, index = False)

## 1.4 variance, only KNN-VDM


In [26]:
# Preparation: Prepare list of metrics for each discretization (test_list)
# Step 1: Obtain variance for each discretization, convert into numeric, string values will be return as NaN
# Filter for Knn-VDM
# Obtain variance for each discretization, convert into numeric, string values will be return as NaN
ewd_acc_knn_vdm = pd.to_numeric(data[(data['disc']=="EWD") & (data['models']=="Knn-VDM")]['variance'],errors='coerce').tolist()
efd_acc_knn_vdm = pd.to_numeric(data[(data['disc']=="EFD") & (data['models']=="Knn-VDM")]['variance'],errors='coerce').tolist()
ffd_acc_knn_vdm = pd.to_numeric(data[(data['disc']=="FFD") & (data['models']=="Knn-VDM")]['variance'],errors='coerce').tolist()
cm_acc_knn_vdm = pd.to_numeric(data[((data['disc']=="ChiMerge") | (data['disc'] == "ChiMerge-SB")) & (data['models']=="Knn-VDM")]['variance'],errors='coerce').tolist()
dt_acc_knn_vdm = pd.to_numeric(data[(data['disc']=="ChiMerge") & (data['models']=="Knn-VDM")]['variance'],errors='coerce').tolist()

# Check number of metrics available for each discretizer
print(len(ewd_acc_knn_vdm))
print(len(efd_acc_knn_vdm))
print(len(ffd_acc_knn_vdm))
print(len(cm_acc_knn_vdm))
print(len(dt_acc_knn_vdm))

# Step 2: filter numeric values
raw_list = [ewd_acc_knn_vdm,efd_acc_knn_vdm, ffd_acc_knn_vdm, cm_acc_knn_vdm, dt_acc_knn_vdm]
num_list = [] # filter numeric values only
for metric in raw_list:
    metric_new = [x for x in metric if (math.isnan(x) == False)] # using list comprehension
    num_list.append(metric_new)
    
# Step 3: random sample
# For discretization methods with different value, randomly select so that the final sample size are equal
# Reference: https://docs.python.org/3/library/random.html
k = min(len(metric) for metric in num_list)
test_list = []
for metric in num_list:
    random.seed(20)
    if len(metric) > k:
        metric = random.sample(metric, k=k)
    else: metric = metric
    #print(metric)
    test_list.append(metric)

9
9
12
8
8


In [27]:
print(num_list)

[[0.035, 0.04, 0.04, 0.092, 0.092, 0.32], [0.071, 0.044, 0.037, 0.087, 0.087, 0.087, 0.155, 0.134, 0.128], [0.011, 0.04, 0.189, 0.189, 0.077, 0.089, 0.086, 0.082, 0.204, 0.138, 0.121, 0.325], [0.01, 0.041, 0.131, 0.126, 0.126], [0.01, 0.041, 0.131, 0.126, 0.126]]


In [28]:
print(k)

5


In [29]:
print(test_list)

[[0.32, 0.04, 0.04, 0.092, 0.035], [0.037, 0.087, 0.087, 0.155, 0.071], [0.325, 0.121, 0.189, 0.077, 0.04], [0.01, 0.041, 0.131, 0.126, 0.126], [0.01, 0.041, 0.131, 0.126, 0.126]]


In [30]:
print(len(test_list))

5


In [31]:
# Initialiation
disc_key = ['ewd', 'efd', 'ffd', 'cm', 'dt']
disc_value = test_list # list of metrics for each discretization after preparation
test_stat = []
p_value = []
disc_compare = []
# Create dictionary store discretization and series of variance
disc = {}
for key in disc_key:
    for value in disc_value:
        disc[key] = value
        disc_value.remove(value)
        break  

In [32]:
# Create loop for Wilcoxon test (two sided)
for i in disc_key:
    for j in disc_key:
        if i != j:
            disc_compare.append(f'{i} vs {j}')
            # print(f'{i} vs {j}')
            # Compute difference list
            diff_list = []
            for m in range(0, len(disc[i])):
                diff = disc[i][m] - disc[j][m]
                diff_list.append(diff)
                                
            if all(item == 0 for item in diff_list) == False: # if the diff list does not contain all 0
                test_stat.append(stats.wilcoxon(diff_list).statistic)
                p_value.append(stats.wilcoxon(diff_list).pvalue)
            else: # if the diff list contain only0, cannot do Wilcoxon test
                error = 'N/A'
                test_stat.append(error)
                p_value.append(error)

In [33]:
# Convert 3 lists to dataframe
disc_compare = pd.DataFrame(disc_compare, columns=['disc_compare'])
test_stat = pd.DataFrame(test_stat, columns=['wtest_stat'])
p_value = pd.DataFrame(p_value, columns=['p_value'])

In [34]:
# Result table
wt_result = pd.concat([disc_compare, test_stat, p_value], axis = 1)
# Drop duplicate
wt_result.drop_duplicates(subset=['p_value'], inplace=True)

In [35]:
wt_result

,disc_compare,wtest_stat,p_value
0,ewd vs efd,5.0,0.625
1,ewd vs ffd,3.0,0.3125
5,efd vs ffd,4.0,0.4375
6,efd vs cm,7.0,1.0
15,cm vs dt,N/A,N/A


In [36]:
# Exporting result

model = 'KNN-VDM'
metric = 'variance'
test = 'two_sided'

filename = f"wilcoxon_{metric}_{model}-{test}.csv"
wt_result['model'] = model
wt_result['metric'] = metric

wt_result.to_csv(filename, index = False)

## 1.5 variance, only KNN-Hamming

Knn-Hamming is applied for 5 datasets during Capita Selecta phases (post BNAIC)


In [37]:
# Preparation: Prepare list of metrics for each discretization (test_list)
# Step 1: Obtain variance for each discretization, convert into numeric, string values will be return as NaN
# Filter for Knn-VDM
# Obtain variance for each discretization, convert into numeric, string values will be return as NaN
ewd_acc_knn_hamming = pd.to_numeric(data[(data['disc']=="EWD") & (data['models']=="Knn-Hamming")]['variance'],errors='coerce').tolist()
efd_acc_knn_hamming = pd.to_numeric(data[(data['disc']=="EFD") & (data['models']=="Knn-Hamming")]['variance'],errors='coerce').tolist()
ffd_acc_knn_hamming = pd.to_numeric(data[(data['disc']=="FFD") & (data['models']=="Knn-Hamming")]['variance'],errors='coerce').tolist()
cm_acc_knn_hamming = pd.to_numeric(data[((data['disc']=="ChiMerge") | (data['disc'] == "ChiMerge-SB")) & (data['models']=="Knn-Hamming")]['variance'],errors='coerce').tolist()
dt_acc_knn_hamming = pd.to_numeric(data[(data['disc']=="ChiMerge") & (data['models']=="Knn-Hamming")]['variance'],errors='coerce').tolist()

# Check number of metrics available for each discretizer
print(len(ewd_acc_knn_hamming))
print(len(efd_acc_knn_hamming))
print(len(ffd_acc_knn_hamming))
print(len(cm_acc_knn_hamming))
print(len(dt_acc_knn_hamming))

# Step 2: filter numeric values
raw_list = [ewd_acc_knn_hamming,efd_acc_knn_hamming, ffd_acc_knn_hamming, cm_acc_knn_hamming, dt_acc_knn_hamming]
num_list = [] # filter numeric values only
for metric in raw_list:
    metric_new = [x for x in metric if (math.isnan(x) == False)] # using list comprehension
    num_list.append(metric_new)
    
# Step 3: random sample
# For discretization methods with different value, randomly select so that the final sample size are equal
# Reference: https://docs.python.org/3/library/random.html
k = min(len(metric) for metric in num_list)
test_list = []
for metric in num_list:
    random.seed(20)
    if len(metric) > k:
        metric = random.sample(metric, k=k)
    else: metric = metric
    #print(metric)
    test_list.append(metric)

21
21
28
32
12


In [38]:
# Initialiation
disc_key = ['ewd', 'efd', 'ffd', 'cm', 'dt']
disc_value = test_list # list of metrics for each discretization after preparation
test_stat = []
p_value = []
disc_compare = []
# Create dictionary store discretization and series of variance
disc = {}
for key in disc_key:
    for value in disc_value:
        disc[key] = value
        disc_value.remove(value)
        break 
        
# Create loop for Wilcoxon test (two sided)
for i in disc_key:
    for j in disc_key:
        if i != j:
            disc_compare.append(f'{i} vs {j}')
            # print(f'{i} vs {j}')
            # Compute difference list
            diff_list = []
            for m in range(0, len(disc[i])):
                diff = disc[i][m] - disc[j][m]
                diff_list.append(diff)
            if all(item == 0 for item in diff_list) == False: # if the diff list does not contain all 0
                test_stat.append(stats.wilcoxon(diff_list).statistic)
                p_value.append(stats.wilcoxon(diff_list).pvalue)
            else: # if the diff list contain only0, cannot do Wilcoxon test
                error = 'N/A'
                test_stat.append(error)
                p_value.append(error)


/Users/anhtth/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/morestats.py:3141: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


In [39]:
# Convert 3 lists to dataframe
disc_compare = pd.DataFrame(disc_compare, columns=['disc_compare'])
test_stat = pd.DataFrame(test_stat, columns=['wtest_stat'])
p_value = pd.DataFrame(p_value, columns=['p_value'])

In [40]:
# Result table
wt_result = pd.concat([disc_compare, test_stat, p_value], axis = 1)
# Drop duplicate
wt_result.drop_duplicates(subset=['p_value'], inplace=True)

In [41]:
wt_result

,disc_compare,wtest_stat,p_value
0,ewd vs efd,29.0,0.722108
1,ewd vs ffd,6.0,0.006836
2,ewd vs cm,24.0,0.266113
5,efd vs ffd,13.0,0.042480
6,efd vs cm,26.0,0.339355
7,efd vs dt,30.0,0.518555
10,ffd vs cm,33.0,0.677246


In [42]:
# Exporting result

model = 'KNN-Hamming'
metric = 'variance'
test = 'two_sided'

filename = f"wilcoxon_{metric}_{model}-{test}.csv"
wt_result['model'] = model
wt_result['metric'] = metric

wt_result.to_csv(filename, index = False)

## 1.6 variance, KNN (Knn-VDM and Knn-Hamming)

Merge both result from Knn-VDM and KNN-Hamming


In [43]:
# Preparation: Prepare list of metrics for each discretization (test_list)
# Step 1: Obtain variance for each discretization, convert into numeric, string values will be return as NaN
# Filter for Knn-VDM
# Obtain variance for each discretization, convert into numeric, string values will be return as NaN
ewd_acc_knn = pd.to_numeric(data[(data['disc']=="EWD") & ((data['models']=="Knn-Hamming") | (data['models']=="Knn-VDM"))]['variance'],errors='coerce').tolist()
efd_acc_knn = pd.to_numeric(data[(data['disc']=="EFD") & ((data['models']=="Knn-Hamming") | (data['models']=="Knn-VDM"))]['variance'],errors='coerce').tolist()
ffd_acc_knn = pd.to_numeric(data[(data['disc']=="FFD") & ((data['models']=="Knn-Hamming") | (data['models']=="Knn-VDM"))]['variance'],errors='coerce').tolist()
cm_acc_knn = pd.to_numeric(data[((data['disc']=="ChiMerge") | (data['disc'] == "ChiMerge-SB")) & ((data['models']=="Knn-Hamming") | (data['models']=="Knn-VDM"))]['variance'],errors='coerce').tolist()
dt_acc_knn = pd.to_numeric(data[(data['disc']=="ChiMerge") & ((data['models']=="Knn-Hamming") | (data['models']=="Knn-VDM"))]['variance'],errors='coerce').tolist()

# Check number of metrics available for each discretizer
print(len(ewd_acc_knn))
print(len(efd_acc_knn))
print(len(ffd_acc_knn))
print(len(cm_acc_knn))
print(len(dt_acc_knn))

# Step 2: filter numeric values
raw_list = [ewd_acc_knn,efd_acc_knn, ffd_acc_knn, cm_acc_knn, dt_acc_knn]
num_list = [] # filter numeric values only
for metric in raw_list:
    metric_new = [x for x in metric if (math.isnan(x) == False)] # using list comprehension
    num_list.append(metric_new)
    
# Step 3: random sample
# For discretization methods with different value, randomly select so that the final sample size are equal
# Reference: https://docs.python.org/3/library/random.html
k = min(len(metric) for metric in num_list)
test_list = []
for metric in num_list:
    random.seed(20)
    if len(metric) > k:
        metric = random.sample(metric, k=k)
    else: metric = metric
    #print(metric)
    test_list.append(metric)

30
30
40
40
20


In [44]:
# Initialiation
disc_key = ['ewd', 'efd', 'ffd', 'cm', 'dt']
disc_value = test_list # list of metrics for each discretization after preparation
test_stat = []
p_value = []
disc_compare = []
# Create dictionary store discretization and series of variance
disc = {}
for key in disc_key:
    for value in disc_value:
        disc[key] = value
        disc_value.remove(value)
        break 
        
# Create loop for Wilcoxon test (two sided)
for i in disc_key:
    for j in disc_key:
        if i != j:
            disc_compare.append(f'{i} vs {j}')
            # print(f'{i} vs {j}')
            # Compute difference list
            diff_list = []
            for m in range(0, len(disc[i])):
                diff = disc[i][m] - disc[j][m]
                diff_list.append(diff)
            if all(item == 0 for item in diff_list) == False: # if the diff list does not contain all 0
                test_stat.append(stats.wilcoxon(diff_list).statistic)
                p_value.append(stats.wilcoxon(diff_list).pvalue)
            else: # if the diff list contain only0, cannot do Wilcoxon test
                error = 'N/A'
                test_stat.append(error)
                p_value.append(error)


/Users/anhtth/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/morestats.py:3141: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


In [45]:
# Convert 3 lists to dataframe
disc_compare = pd.DataFrame(disc_compare, columns=['disc_compare'])
test_stat = pd.DataFrame(test_stat, columns=['wtest_stat'])
p_value = pd.DataFrame(p_value, columns=['p_value'])

In [46]:
# Result table
wt_result = pd.concat([disc_compare, test_stat, p_value], axis = 1)
# Drop duplicate
wt_result.drop_duplicates(subset=['p_value'], inplace=True)
wt_result

,disc_compare,wtest_stat,p_value
0,ewd vs efd,46.0,0.159378
1,ewd vs ffd,43.0,0.120209
2,ewd vs cm,42.0,0.178810
3,ewd vs dt,73.0,0.889969
5,efd vs ffd,21.0,0.006653
6,efd vs cm,25.0,0.026184
7,efd vs dt,54.0,0.306046
10,ffd vs cm,65.0,0.611221
11,ffd vs dt,52.0,0.263321
15,cm vs dt,38.5,0.127094


In [47]:
# Exporting result

model = 'KNN'
metric = 'variance'
test = 'two_sided'

filename = f"wilcoxon_{metric}_{model}-{test}.csv"
wt_result['model'] = model
wt_result['metric'] = metric

wt_result.to_csv(filename, index = False)